In [1]:
import pandas as pd
import random

# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

nb_phrase_lang = 40000
df_big = create_lang_df('../data/multilingue/sentences.csv')
df_en = df_big[df_big['lan_code']=='eng'].iloc[:nb_phrase_lang]
df_fr = df_big[df_big['lan_code']=='fra'].iloc[:nb_phrase_lang]
df_de = df_big[df_big['lan_code']=='deu'].iloc[:nb_phrase_lang]
df_es = df_big[df_big['lan_code']=='spa'].iloc[:nb_phrase_lang]
df_it = df_big[df_big['lan_code']=='ita'].iloc[:nb_phrase_lang]
df = pd.concat([df_en,df_fr,df_de,df_es,df_it]).sample(frac=1, random_state=42).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
df

Nombre de lignes de sentence.csv: 200000


,lan_code,sentence
0,deu,Der Hund wedelte mit dem Schwanz und folgte se...
1,fra,Tu devrais téléphoner à ta mère le plus tôt po...
2,spa,No es para tomarse a risa que él no se haya po...
3,fra,Ils arrivèrent les uns après les autres.
4,eng,That custom originated with the American Indians.
...,...,...
199995,deu,Was genau ist denn eigentlich an dieser sogena...
199996,deu,Putz dir mindestens zweimal pro Tag die Zähne.
199997,spa,¿A qué se dedica tu papá?
199998,spa,Los grandes artistas jamás ven las cosas tal c...


In [2]:
# Créez un vocabulaire complet à partir des données anglaises et française
from transformers import BertTokenizerFast
from collections import Counter

nb_token_max = 2000
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')
# Création d'un dictionnaire de token
vocab = Counter(token for ligne in df['sentence'] for token in tokenizer.tokenize(ligne))
# Tri des token en fonction de leur fréquence
vocab= sorted(vocab.items(), key=lambda x: x[1], reverse=True) 
# Limitation du nombre de token
vocab = dict(vocab[:nb_token_max])


In [3]:
# créer 2 dataframes: 1 train (70% des phrases) et 1 test (30% des phrases)
n_train = int(n_rows*0.7)
df_train = df.iloc[:n_train].sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df.iloc[n_train:].sample(frac=1, random_state=24).reset_index(drop=True)
pd.set_option('display.max_colwidth', 150)
display(df_train)

,lan_code,sentence
0,deu,Er schrieb das Buch im Alter von zwanzig.
1,eng,"I'd like to invite you to lunch, if you're not busy."
2,fra,Qui est cette dame ?
3,eng,It is finally all over. Now we can relax.
4,deu,Was steht auf dem Schild?
...,...,...
139995,fra,« Quand vas-tu revenir ? » « Tout dépend du temps qu'il fera. »
139996,ita,Discutiamo di questo problema.
139997,deu,Ich lerne zwei Fremdsprachen.
139998,fra,UN signifie Nations Unies.


In [12]:
import numpy as np

# Créez un DataFrame BOW avec les phrases (lignes) et les fréquences de chaque token (colonnes)
def create_BOW(data):
    BOW = []
    for ligne in data:
        l_tokenised = tokenizer.tokenize(ligne)
        BOW.append([l_tokenised.count(token) for token in vocab])
    return BOW
            
X_train = create_BOW(df_train['sentence'])
y_train = df_train['lan_code'].values.tolist() 

#
X_test = create_BOW(df_test['sentence'])
y_test = df_test['lan_code'].values.tolist()


In [13]:
from sklearn import svm
from sklearn import naive_bayes

# On entraine et on prédit
clf_nb = naive_bayes.BernoulliNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
pd.crosstab(y_test,y_pred_nb,rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,deu,eng,fra,ita,spa
Classe réelle,,,,,
deu,12068,7,4,17,5
eng,5,12041,2,19,13
fra,5,8,11798,70,72
ita,2,15,13,11612,86
spa,6,7,12,97,12016


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Créer un classificateur clf et entraîner le modèle sur l'ensemble d'entraînement
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# Calculer les prédictions 
y_pred_gb = clf_gb.predict(X_test)
pd.crosstab(y_test,y_pred_gb,rownames=['Classe réelle'], colnames=['Classe prédite'])

In [8]:

# Instanciation d'un exemple
exemple = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "vamos a la playa",
           "Ich heiße Keyne, und das ist wunderbar",
           "she loves you, mais elle te hait aussi, and das ist traurig", # Attention à cette phrase trilingue
           "Vogliamo visitare il Colosseo e nuotare nel Tevere."
          ]


# Prédiction des classes de l'exemple par le classifieur Naive Bayes
predictions_nb = clf_nb.predict(create_BOW(exemple))

# Prédiction des classes de l'exemple par le classifieur Gradiant Boosting
predictions_gb = clf_gb.predict(create_BOW(exemple))

# Affichage des prédictions
print('Prédictions Naive Bayes :',predictions_nb)
print('Prédictions Gradiant Boosting :',predictions_gb)

Prédictions Naive Bayes : ['eng' 'fra' 'eng' 'spa' 'deu' 'eng']
Prédictions Gradiant Boosting : ['eng' 'fra' 'eng' 'spa' 'deu' 'eng']
